# CUSTOM MODELS AND TRAINING WITH TENSORFLOW

## USING TENSORFLOW LIKE NUMPY

In [1]:
import tensorflow as tf

In [2]:
### TENSORS AND OPERATIONS
tf.constant([[1., 2., 3.], [4., 5., 6.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

In [5]:
t.shape

TensorShape([2, 3])

In [6]:
t.dtype

tf.float32

In [7]:
t[:, 1:] # indexing

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [9]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [10]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [11]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [12]:
tf.multiply(t,10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[10., 20., 30.],
       [40., 50., 60.]], dtype=float32)>

In [13]:
tf.reduce_mean(t)

<tf.Tensor: shape=(), dtype=float32, numpy=3.5>

### TENSORS AND NUMPY

In [14]:
import numpy as np
a = np.array([2., 4. , 5.])

In [15]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [16]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [17]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [18]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### TYPE CONVERSIONS

In [19]:
t2 = tf.constant(40., dtype = tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### VARIABLES

In [20]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [21]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [22]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [23]:
v[:, 2].assign([0.,1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [24]:
v.scatter_nd_update(indices = [[0,0], [1,2]], updates = [100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

## CUSTOM MODELS AND TRAINING ALGORTITHMS

### CUSTOM LOSS FUNCTIONS

In [25]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [26]:
from tensorflow import keras
import pandas as pd

In [27]:
from sklearn.datasets import load_iris
iris = load_iris()
x, y = iris["data"], iris["target"]
y = pd.to_numeric(y, downcast = "float")

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42, train_size = 0.8)

In [29]:
input_A = keras.layers.Input(shape = 4)
hidden_A = keras.layers.Dense(30, activation = "relu")(input_A)
hidden_B = keras.layers.Dense(30, activation = "relu")(hidden_A)
concat = keras.layers.concatenate([input_A, hidden_B])
output = keras.layers.Dense(1, activation = "softmax")(concat)
model = keras.Model(inputs = [input_A], outputs = [output])

In [30]:
model.compile(loss = huber_fn, optimizer = "nadam")

In [31]:
x_train_n, x_val, y_train_n, y_val = train_test_split(x_train, y_train)

In [32]:
new_model = model.fit(x_train_n, y_train_n, epochs = 10, validation_data = (x_val, y_val))

Train on 90 samples, validate on 30 samples
Epoch 1/10
90/90 [==============================] - 5s 50ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 2/10
90/90 [==============================] - 3s 37ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 3/10
90/90 [==============================] - 3s 33ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 4/10
90/90 [==============================] - 4s 41ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 5/10
90/90 [==============================] - 3s 37ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 6/10
90/90 [==============================] - 4s 42ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 7/10
90/90 [==============================] - 3s 35ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 8/10
90/90 [==============================] - 4s 39ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 9/10
90/90 [==============================] - 3s 34ms/sample - loss: 0.3167 - val_loss: 0.3667
Epoch 10/10
90/90 [============================

In [33]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [34]:
model.compile(loss = create_huber(2.0), optimizer = "nadam")

In [35]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2/ 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

#### Custom Activation Functions, Initializers, Regularizers, and Constraints

In [36]:
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1.0)
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))
def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [37]:
layer = keras.layers.Dense(30, activation=my_softplus, kernel_initializer=my_glorot_initializer, kernel_regularizer=my_l1_regularizer, kernel_constraint=my_positive_weights)

In [38]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

#### Custom Metrics

In [39]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

In [40]:
precision = keras.metrics.Precision()

In [41]:
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [42]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

#### Custom Layers

In [43]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [44]:
## Redefining the dense layer
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
        name="kernel", shape=[batch_input_shape[-1], self.units], initializer="glorot_normal")
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, "activation": keras.activations.serialize(self.activation)}

In [45]:
## Creating a layer with multiple inputs like concatenate
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / X2]
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1] # should probably handle broadcasting rules

In [46]:
## Creating a layer that adds gaussian noise
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

#### Custom Models

In [47]:
## Building a residual block
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal") for _ in range(n_layers)]
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [48]:
class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
        kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
            Z = self.block2(Z)
        return self.out(Z)

#### Losses and Metrics based on Model Internals

In [49]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal") for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)

#### Computing Gradients using AutoDiff

In [50]:
def f(w1,w2):
    return 3*w1**2 + 2*w1*w2

In [51]:
w1, w2 = 5,3
eps = 1e-6
print((f(w1+eps,w2)-f(w1,w2))/eps)
print((f(w1,w2+eps)-f(w1,w2))/eps)

36.000003007075065
10.000000003174137


In [52]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1,w2)
gradients = tape.gradient(z, [w1,w2])

In [53]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [54]:
with tf.GradientTape(persistent=True) as tape: ## by setting persistent to True, it can be ran multiple times
    z = f(w1,w2)
dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)
print(dz_dw1, ", ", dz_dw2)

tf.Tensor(36.0, shape=(), dtype=float32) ,  tf.Tensor(10.0, shape=(), dtype=float32)


In [55]:
del tape

In [56]:
## An attempt to use any other thing other than variables yield None
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1, c2)
tape.gradient(z, [c1, c2]) # returns [None, None] 

[None, None]

In [57]:
## tape watch forces tape to use any kinds of tensors
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)
tape.gradient(z, [c1, c2]) # returns [tensor 36., tensor 10.]

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [58]:
## tf.stop_gradient prevents backpropagation
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)
with tf.GradientTape() as tape:
    z = f(w1, w2) # same result as without stop_gradient()
tape.gradient(z, [w1, w2])

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

In [59]:
## to stabilize the softplus approach and ensure the gradients doesn't return NaN
@tf.custom_gradient
def my_better_softplus(z):
    exp = tf.exp(z)
    def my_softplus_gradients(grad):
        return grad / (1 + 1 / exp)
    return tf.math.log(exp + 1), my_softplus_gradients

#### Custom Training Loop

In [60]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([keras.layers.Dense(30, activation = "elu", kernel_initializer = "he_normal", kernel_regularizer = l2_reg),
                                 keras.layers.Dense(1, kernel_regularizer = l2_reg)])

In [61]:
def random_batch(X, y, batch_size = 32):
    idx = np.random.randint(len(X), size = batch_size)
    return X[idx], y[idx]

In [62]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)


In [63]:
n_epochs = 5
batch_size = 32
n_steps = len(x_train)//batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [64]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(x_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
    for metric in metrics:
        metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss,metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

120/120 - mean: 48.5687 - mean_absolute_error: 5.9935
Epoch 2/5
120/120 - mean: 26.2081 - mean_absolute_error: 4.0712
Epoch 3/5
120/120 - mean: 8.4342 - mean_absolute_error: 2.1310
Epoch 4/5
120/120 - mean: 5.6094 - mean_absolute_error: 1.8408
Epoch 5/5
120/120 - mean: 3.7176 - mean_absolute_error: 1.1148


#### TensorFlow Functions and Graphs

In [65]:
def cube(x):
    return x**3

In [66]:
cube(2)

8

In [67]:
cube(tf.constant([2.0]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.], dtype=float32)>

In [68]:
tf_cube = tf.function(cube)
tf_cube

In [69]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [70]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [71]:
@tf.function
def tf_cube(x):
    return x**3

In [72]:
tf_cube.python_function(2)

8